Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:
import tensorflow as tf
from tensorflow import keras

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 15s 1us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28) ),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation = tf.nn.softmax)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.
 * relu says if x > 0 retun x else return 0
 * softmax pic the biggest number in the output

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[0.13118318, 0.12794341, 0.17853579, 0.05131614, 0.05832292,
        0.14751516, 0.04726008, 0.07319599, 0.08421265, 0.10051475]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.10307398, 0.10274059, 0.10807221, 0.09516193, 0.09583105,
        0.1047712 , 0.09477673, 0.097267  , 0.09834448, 0.09996084]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.2559764

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 1.5851 - accuracy: 0.8951
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 1.5215 - accuracy: 0.9460
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.5085 - accuracy: 0.9575
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5009 - accuracy: 0.9639
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 1.4964 - accuracy: 0.9675


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533674, 0.08533674, 0.08533674, 0.08533674, 0.08533674,
        0.08533674, 0.08533674, 0.23196931, 0.08533674, 0.08533674],
       [0.08533674, 0.08533674, 0.23196931, 0.08533674, 0.08533674,
        0.08533674, 0.08533674, 0.08533674, 0.08533674, 0.08533674],
       [0.08533697, 0.23196565, 0.08533703, 0.08533698, 0.08533697,
        0.08533698, 0.08533766, 0.0853372 , 0.08533756, 0.08533697],
       [0.23196934, 0.08533675, 0.08533675, 0.08533675, 0.08533675,
        0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.08533675],
       [0.08533718, 0.08533718, 0.08533718, 0.08533718, 0.23196238,
        0.08533718, 0.08533718, 0.08533737, 0.08533718, 0.08533998]],
      dtype=float32)>

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 1.4956 - accuracy: 0.9680


[1.4955635070800781, 0.9679999947547913]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).